In [2]:
## Purpose: Clip listed datasets to input area 

In [3]:
# How to get file list
# 1. Use data downloads lookups
# 2. Get list of all files in an area
# 3. Create own list

In [4]:
import pandas as pd
import geopandas as gpd
from pathlib import Path
import os
from datetime import date

In [5]:
#### Make changes
# What boundary are you clipping too?
# Is it a parish council (answer yes or no)
parish_council = "yes"
# If yes, which parish council
boundary_name = "Market Lavington"

# If no, what is the filepath to the boundary?
boundary_area_filepath = "O:/Data_team/GIS_data_downloads/CountyBoundary.shp"
if parish_council != "yes":
    # And what is the boundary name?
    boundary_name = "Wiltshire"

# Do you want a buffer around the boundary?
buffer = "yes"
# If yes, what size buffer?
buffer_size = 2000

In [6]:
# Set boundary
if parish_council == "yes":
    pc_boundaries_filepath = "G:/OS_OpenData/OS_BoundaryLine/OS_BL_Parish_Wilts.shp"
    pc_boundaries_gdf = gpd.read_file(pc_boundaries_filepath)
    boundary_area = pc_boundaries_gdf[pc_boundaries_gdf.NAME == boundary_name]

else:
    boundary_area = gpd.read_file(boundary_area_filepath)
#boundary_area_filepath = "O:/Data_team/GIS_data_downloads/CountyBoundary.shp"

if buffer == "yes":
    boundary_area.geometry = boundary_area.geometry.buffer(buffer_size)

c:\Temp\venvs\data-downloads\Lib\site-packages\geopandas\geodataframe.py:1968: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [7]:
# location to write clipped data to
download_location = "O:/Data_team/GIS_data_downloads/dataPackage/"

# Find all datasets
dataset_area = Path("G:/GIS_Data/External/Open_Source/Original")
dataset_list = list(dataset_area.rglob("*.gpkg"))

# Set date for file name
today = date.today().strftime("%d_%m_%Y")

In [9]:
for dataset_path in dataset_list:
    filename = Path(dataset_path).stem

    # Loop through each layer in file, clip to wilts and write to new location

    # List layers in geopackage
    gpd_layers = gpd.list_layers(dataset_path)
    # Need separate code for boundary line, as handle each layer in geopackage differently (clip or interesect)
    if filename in ["BoundaryLine", "OS_Open_Zoomstack"]:
        pass
    else:
        count = 0
        for i, layer in enumerate(gpd_layers.name):
            print(filename, layer)
            # Read in each layer of dataset
            gdf = gpd.read_file(dataset_path, layer = layer)
            # Clip dataset
            gdf_clipped = gpd.clip(gdf, boundary_area)
            # Check that geodataframe contains data
            if gdf_clipped.empty == False:
                # Write or append geopackage layer
                gdf_clipped.to_file(f"{download_location}{boundary_name}/{filename}_{today}.gpkg",layer = layer, driver = "GPKG", mode = "w" if count==0 else "a")
                count=+1

    # # Read in dataset
    # gdf = gpd.read_file(dataset_path)

    # # Clip dataset
    # gdf_clipped = gpd.clip(gdf, boundary_area)

    # # Write out dataset
    # if not os.path.exists(f"{download_location}{boundary_name}"):
    #     os.makedirs(f"{download_location}{boundary_name}")

    # # Only write out file if it contains something
    # if gdf_clipped.empty == False:
    #     gdf_clipped.to_file(f"{download_location}{boundary_name}/{filename}_{today}.gpkg",layer=f"{filename}", driver="GPKG")
    #     #gdf_clipped.to_file(f"{download_location}{boundary_name}/{filename}.shp")

Ancient Tree Inventory (ATI) Ancient Tree Inventory (ATI)
National Cycle Network (Public) National Cycle Network (Public)
Reclassified Routes (Public) Reclassified Routes (Public)
IBAs UK IBAs UK
RSPB Priority Landscapes RSPB Priority Landscapes
RSPB Reserves RSPB Reserves
os_open_built_up_areas os_open_built_up_areas
os_open_built_up_areas os_open_built_up_extents
os_open_built_up_areas os_open_non_built_up_extents
terr50_gb contour_line
terr50_gb land_water_boundary
terr50_gb spot_height
oproad_gb motorway_junction
oproad_gb road_link
oproad_gb road_node
oprvrs_gb hydro_node
oprvrs_gb watercourse_link
opname_gb named_place
opmplc_gb building
opmplc_gb car_charging_point
opmplc_gb electricity_transmission_line
opmplc_gb foreshore
opmplc_gb functional_site
opmplc_gb glasshouse
opmplc_gb important_building
opmplc_gb motorway_junction
opmplc_gb named_place
opmplc_gb railway_station
opmplc_gb railway_track
opmplc_gb railway_tunnel
opmplc_gb road
opmplc_gb road_tunnel
opmplc_gb roundabout
